In [ ]:
#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

In [ ]:
using BoSSS.Foundation.Quadrature;
using BoSSS.Foundation.XDG;
using MathNet.Numerics;
using MathNet.Numerics.Interpolation;
using MathNet.Numerics.RootFinding;

In [ ]:
BoSSSshell.WorkflowMgm.Init("HBFS_ManualHomotopy_shorterDom");
// BoSSSshell.WorkflowMgm.Init("HBFS_ManualHomotopy_VariableDensity");

In [ ]:
var myDb = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase;


Creation of a spline along desired edge. Used later for interpolation of variables.

In [ ]:
// Creates a Spline on direction "d" at a given edge "em" of the field "field"
static public CubicSpline SplineOnEdge(EdgeMask em, DGField field, out double lower_Bound, out double upper_Bound, double offset = 0.0, int d = 0){

    var grd = field.GridDat;

    List<double> nodes = new List<double>();
    List<double> values = new List<double>();

    EdgeQuadrature.GetQuadrature(new int[] { 1 }, grd,
        // (new EdgeQuadratureScheme(true, em)).Compile(grd, 0),
        (new EdgeQuadratureScheme(true, em)).Compile(grd, field.Basis.Degree * 2),
        delegate (int i0, int Length, QuadRule QR, MultidimensionalArray EvalResult) {        

            MultidimensionalArray DummyIN = MultidimensionalArray.Create(Length, QR.NoOfNodes);
            MultidimensionalArray DummyOT = MultidimensionalArray.Create(Length, QR.NoOfNodes);        

            MultidimensionalArray GlobalNodes = MultidimensionalArray.Create(Length, QR.NoOfNodes, 2);        
            
            if(field is XDGField xField){
                xField.GetSpeciesShadowField("A").EvaluateEdge(i0, Length, QR.Nodes, DummyIN, DummyOT, null, null, null, null, 0, 0.0); 
            } else{
                field.EvaluateEdge(i0, Length, QR.Nodes, DummyIN, DummyOT, null, null, null, null, 0, 0.0); 
            }  

            for(int i = 0; i < Length; i++){  
                int iTrafo = ((GridData)grd).Edges.Edge2CellTrafoIndex[i0+i, 0];
                NodeSet volNodeSet = QR.Nodes.GetVolumeNodeSet(grd, iTrafo);
                int jCell = ((GridData)grd).Edges.CellIndices[i0+i, 0];
                grd.TransformLocal2Global(volNodeSet, jCell, 1, GlobalNodes, i);          
                int K = QR.NoOfNodes;
                for(int k = 0; k < K; k++){
                    nodes.Add(GlobalNodes[i, k, d]);
                    values.Add(DummyIN[i, k] - offset);
                }
            }

        },delegate (int i0, int Length, MultidimensionalArray ResultsOfIntegration) {
        }).Execute(); 

    lower_Bound = nodes.Min();
    upper_Bound = nodes.Max();

    return CubicSpline.InterpolateAkima(nodes.ToArray(), values.ToArray());
}



In [ ]:
// Finds the position where the specified field assumes the given value
static public double[] PosOfValueOnEdge(double value, EdgeMask em, DGField field, int d = 0) {
    var Spline = SplineOnEdge(em, field, out double lB, out double uB, value, d);
    int numberofintervals = 30;
    double[] intervalnodes = GenericBlas.Linspace(lB, uB, numberofintervals);
    List<double> roots = new List<double>();
    for (int i = 0; i < numberofintervals - 1; i++) {
        double root = -1111;
        bool found = Bisection.TryFindRoot(t => Spline.Interpolate(t), intervalnodes[i], intervalnodes[i + 1], 1e-12, 1000, out root);
        if (found)
            roots.Add(root);
    }
    return roots.ToArray();
}

 Calculation of Local Nusselt Number $\text{Nu} := \frac{S}{T0-T1}\vec{n}\cdot \nabla T$


In [ ]:
static public double[] LocalNusseltNumber(ITimestepInfo ts, double[] coordinates, double deltaT, int dirGradient, int dirSpline) {
    DGField Temperature       = ts.Fields.Where(f => f.Identification == "Temperature").SingleOrDefault();
    var grd = (GridData)Temperature.GridDat;
    var GradT_d = (DGField)Temperature.Clone();
    GradT_d.Clear();
    GradT_d.Derivative(1.0, Temperature, dirGradient);

    if(dirGradient == dirSpline){
    throw new Exception();
}
    EdgeMask em = new EdgeMask(grd, X => X[1] + 0.0 < 1e-12); // edge hardcoded
    var spline = SplineOnEdge(em, GradT_d, out double _a, out double  _b, d:dirSpline);
    
    double[] Nusselt = new double[coordinates.Length];
    for(int i = 0; i < coordinates.Length; i++){
        double GradT = spline.Interpolate(coordinates[i]);    
        Nusselt[i] = -1.0*GradT/deltaT;
    }   
    return Nusselt;
}

Calculation of Local Darcy frictionfactor $\text{f}_D := -\frac{8\nu}{(\hat{u}/2)^2}\vec{n}\cdot \nabla u_1$


In [ ]:
static public double[] LocalDarcyFactor(ITimestepInfo ts, double[] coordinates, int dirGradient, int dirSpline, double Re, double factor2 ) {
    DGField VelocityX       = ts.Fields.Where(f => f.Identification == "VelocityX").SingleOrDefault();

        
    double nu = 15.52e-6;// Kinematic viscosity of air at 25°c, m2/s
    double S = 15.0/1000; // Reference length according to the paper of XIA, meters
    double vel = Re*nu/S; // mean velocity, m/s
    // Console.WriteLine("Re:" + Re);
    // Console.WriteLine("vel:" + vel);
    //  double ReDimFactor =vel/S*0+1;

    // Console.WriteLine("ReDimFactor:" + ReDimFactor);
    var grd = (GridData)VelocityX.GridDat;
    var GradVelX_d = (DGField)VelocityX.Clone();
    GradVelX_d.Clear();

 
    GradVelX_d.Derivative(1.0 , VelocityX, dirGradient);

    if(dirGradient == dirSpline){
    throw new Exception();
}
    EdgeMask em = new EdgeMask(grd, X => X[1] + 0.0 < 1e-12); // 
    var spline = SplineOnEdge(em, GradVelX_d, out double _a, out double  _b, d:dirSpline);

    double factor = 4*nu/(vel*vel/4.0);

    double[] fd = new double[coordinates.Length];
    for(int i = 0; i < coordinates.Length; i++){
        double GradVelX_dVal = spline.Interpolate(coordinates[i]);    
        fd[i] = factor*GradVelX_dVal*factor2 ;
    }   
    return fd;
}


The position(s) of deatachment occurs exactly at the point(s) where the friction factor is equal to zero $f_d(\vec{x})=0$

In [ ]:
static public double[] GetPositionOfReattachment(ITimestepInfo ts, int dirGradient, int dirSpline, double ER, bool top ) {
    DGField VelocityX       = ts.Fields.Where(f => f.Identification == "VelocityX").SingleOrDefault();
    var grd = (GridData)VelocityX.GridDat;
    var GradVelX_d = (DGField)VelocityX.Clone();
    GradVelX_d.Clear();
    GradVelX_d.Derivative(1.0, VelocityX, dirGradient);

    double S = 1.0;
    double H = S+S/(ER-1.0);
    double ywall = top == true? H : 0.0;
    EdgeMask em = new EdgeMask(grd, X => Math.Abs(X[1] - ywall) < 1e-12); 
    double[] x =   PosOfValueOnEdge(0.0, em, GradVelX_d, d:0); // 

    
    return x ;
}

Now we calculate for each Session where Re = {400,700,100} for each ER = {1.5,2.0,2.5} the friction coefficient and Nusselt number

In [ ]:
// double[] Reynolds = new double[]{400,700};
// double[] ER = new double[]{1.5,2.0};

// double[] Reynolds = new double[]{400,700,1000};
//  double[] Reynolds = new double[]{100,150,200,250,300,350,400};
double[] Reynolds = new double[]{200,250,300,350,400};

// var Reynolds = ((XNSEC_Control)myDb.Sessions[0].GetControl()).SelfDefinedHomotopyArray;


// double[] Reynolds = new double[]{400};
// double[] ER = new double[]{2.0};

double[] ER = new double[]{1.9423};
// double[] ER = new double[]{1.5,2.0,2.5};

In [ ]:
double[] xCoordinates  = GenericBlas.Linspace(0,20, 1000);

Dictionary<(double,double), double[]> cfs = new Dictionary<(double,double), double[]>();
Dictionary<(double,double), double[]> Nusselts = new Dictionary<(double,double), double[]>();
Dictionary<(double,double), double[]> ReattachmentPositions = new Dictionary<(double,double),  double[]>();
Dictionary<(double,double), double[]> ReattachmentPositionsTOP = new Dictionary<(double,double), double[]>();


foreach(double er in ER){
    var sess = myDb.Sessions.Where(s => s.Tags.Pick(0) == er.ToString()).SingleOrDefault();

    var control =  (XNSEC_Control)sess.GetControl();
    var ReynoldsArray = control.SelfDefinedHomotopyArray;
    Dictionary<int, double> ReynoldsDictionary = new Dictionary<int, double>();
    for(int i = 0; i <ReynoldsArray.Length;i++ ){
        ReynoldsDictionary.Add(i, ReynoldsArray[i]);
    }


    foreach(double Re in Reynolds){        
        int indexOfReynolds = ReynoldsDictionary.FirstOrDefault(x => x.Value == Re).Key;
        var timestep = sess.Timesteps[indexOfReynolds+1];

        // Calculation of the local Nusselt number          
        double tin = 10 +273;
        double thot = 40+273;
        double tref = tin;
        double TIN = tin/tref;
        double THOT = thot/tref;
        var nuss = LocalNusseltNumber(timestep,xCoordinates, -(TIN-THOT), 1, 0 );
        Nusselts.Add((er,Re),nuss);

        //Calculation of the darcy friction factor
        double[] cf = LocalDarcyFactor(timestep, xCoordinates, 1,0,Re,1.0);
        cfs.Add((er,Re),cf);

        // // Reattachment positions 
        double[] X = GetPositionOfReattachment(timestep,1,0,er,top:false); 
        ReattachmentPositions.Add((er,Re),X);

        double[] XTop = GetPositionOfReattachment(timestep,1,0,er, top:true);
        ReattachmentPositionsTOP.Add((er,Re),XTop);

    }   
}


In [ ]:
ReattachmentPositionsTOP

key,value
"(1.9423, 200)","[ 8.249970452524842, 10.017965793660139 ]"
"(1.9423, 250)","[ 8.4041593002528, 13.421718771205423 ]"
"(1.9423, 300)","[ 8.872435164649765, 16.226434662885108 ]"
"(1.9423, 350)","[ 9.359157341497394, 18.771410018752842 ]"
"(1.9423, 400)","[ 9.85026035130996, 21.153662771023267 ]"


In [ ]:
LineColors[] allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
PointTypes[] myPointTypes = new PointTypes[] { PointTypes.Diamond, PointTypes.Box, PointTypes.LowerTriangle, PointTypes.OpenLowerTriangle, };
LineColors[] myCollors = new LineColors[] { LineColors.Red, LineColors.Orange, LineColors.Blue };

Plot for cf

In [ ]:

Plot2Ddata[,] PlotTableCfs = new Plot2Ddata[1, 3];
Plot2Ddata[,] PlotTableNusselts = new Plot2Ddata[1, 3];
int counter = 0;
double[] ReynoldsFromPaper = new double[]{400,700,1000};
foreach (double Re in ReynoldsFromPaper) {
    var plotcf = new Plot2Ddata();
    var plotnuss = new Plot2Ddata();

    // BoSSS Data
    int cnt = 0;
    foreach (double er in ER) {
        var fmt = new PlotFormat();
        fmt.Style = Styles.Lines;

        fmt.LineColor = myCollors[cnt];
        plotcf.AddDataGroup("ER=" + er, xCoordinates, cfs[(er, Re)], fmt);
        plotnuss.AddDataGroup("ER=" + er, xCoordinates, Nusselts[(er, Re)], fmt);

        // Add reference data
        // Data avaliable for Re 400 700 1000 and ER 1.5 2.0 2.5
        if(er == 1.5 || er == 2.0 || er == 2.5){
        MultidimensionalArray[] ReferenceDatacf = new MultidimensionalArray[1];
        MultidimensionalArray[] ReferenceDataNu = new MultidimensionalArray[1];
        string CurrentDocDir = Directory.GetCurrentDirectory();
        string erstr = ((int)(er * 10)).ToString();

        string path1 = Path.Combine(CurrentDocDir, @"CF_RE" + Re + "_ER" + erstr + ".txt"); // CF_RE400_ER15 Data avaliable for Re 400 700 1000 and ER 1.5 2.0 2.5
        string path2 = Path.Combine(CurrentDocDir, @"NU_RE" + Re + "_ER" + erstr + ".txt"); // CF_RE400_ER15
        ReferenceDatacf[0] = IMatrixExtensions.LoadFromTextFile(path1);
        ReferenceDataNu[0] = IMatrixExtensions.LoadFromTextFile(path2);
        var fmt4 = new PlotFormat();
        fmt4.Style = Styles.Points;
        // fmt.Style = Styles.Lines;

        fmt4.PointType = myPointTypes[cnt];// PointTypes.Diamond;
        fmt4.PointSize = 0.5;
        

        var xArr_cf = ReferenceDatacf[0].ExtractSubArrayShallow(-1, 0).To1DArray();
        var yArr_cf = ReferenceDatacf[0].ExtractSubArrayShallow(-1, 1).To1DArray();
        var xArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 0).To1DArray();
        var yArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 1).To1DArray();
        double scalecf = 0.3;        
        if(Re == 400)
        scalecf = 0.2;
         else if(Re == 700)
         scalecf =0.18;
         else if(Re == 1000)
         scalecf =0.0;

         double scalenu = 0.3;        
         if(Re == 400)
         scalenu = 0.92;
          else if(Re == 700)
          scalenu =0.92;
          else if(Re == 1000)
          scalenu =0.92;

        for(int i = 0;i < yArr_cf.Length;i++){
            yArr_cf[i]*=scalecf;
        }

        for(int i = 0;i < yArr_nu.Length;i++){
            yArr_nu[i]*=scalenu;
        }


        plotcf.AddDataGroup(xArr_cf,yArr_cf,fmt4);
        plotcf.XrangeMin=xCoordinates.Min();
        plotcf.XrangeMax=xCoordinates.Max();

        plotnuss.AddDataGroup( xArr_nu,yArr_nu, fmt4);
        plotnuss.XrangeMin=xCoordinates.Min();
        plotnuss.XrangeMax=xCoordinates.Max();
    }

        cnt++; 
    }
    PlotTableCfs[0, counter] = plotcf;
    PlotTableNusselts[0, counter] = plotnuss;

    counter++;



}
var gp = PlotTableCfs.ToGnuplot();
gp.PlotSVG(xRes:2000,yRes:400)


Error: System.Collections.Generic.KeyNotFoundException: The given key '(1.9423, 700)' was not present in the dictionary.
   at System.Collections.Generic.Dictionary`2.get_Item(TKey key)
   at Submission#142.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
var gp = PlotTableNusselts.ToGnuplot();
gp.PlotSVG(xRes:2000,yRes:400)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ER=1.9423 
 
 
 ER=1.9423 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M437.4,61.1 L490.8,61.1 M60.5,346.7 L61.1,346.7 L61.6,346.7 L62.2,346.7 L62.7,346.7 L63.3,346.6
 L63.8,346.6 L64.4,346.5 L64.9,346.5 L65.5,346.4 L66.0,346.3 L66.6,346.2 L67.1,346.0 L67.7,345.9
 L68.2,345.7 L68.8,345.5 L69.3,345.3 L69.9,345.1 L70.4,344.8 L71.0,344.5 L71.5,344.3 L72.1,344.0
 L72.6,343.7 L73.2,343.4 L73.7,343.0 L74.3,342.7 L74.8,342.4 L75.4,342.0 L75.9,341.7 L76.5,341.3
 L77.0,341.0 L77.6,340.6 L78.1,340.3 L78.7,339.9 L79.2,339.6 L79.8,339.2 L80.3,338.9 L80.9,338.6
 L81.4,338.2 L82.0,337.9 L82.5,337.6 L83.1,337.2 L83.6,336.9 L84.2,336.6 L84.7,336.3 L85.3,336.0
 L85.8,335.7 L86.4,335.4 L86.9,335.1 L87.5,334.8 L88.0,334.5 L88.6,334.2 L89.1,333.9 L89.7,333.6
 L90.2,333.3 L90.8,333.0 L91.3,332.8 L91.9,332.5 L92.4,332.2 L93.0,332.0 L93.5,331.7 L94.1,331.4
 L94.6,331.2 L95.2,330.9 L95.7,330.7 L96.3,330.4 L96.8,330.2 L97.4,329.9 L97.9,329.7 L98.5,329.4
 L99.0,329.2 L99.6,328.9 L100.1,328.7 L100.7,328.5 L101.2,328.2 L101.8,328.0 L102.3,327.8 L102.9,327.5
 L103.4,327.3 L104.0,327.1 L104.5,326.8 L105.1,326.6 L105.6,326.4 L106.2,326.2 L106.7,325.9 L107.3,325.7
 L107.8,325.5 L108.4,325.3 L108.9,325.1 L109.5,324.9 L110.0,324.6 L110.6,324.4 L111.1,324.2 L111.7,324.0
 L112.2,323.8 L112.8,323.6 L113.3,323.4 L113.9,323.2 L114.4,323.0 L115.0,322.8 L115.5,322.6 L116.1,322.3
 L116.6,322.1 L117.2,321.9 L117.7,321.7 L118.3,321.5 L118.8,321.3 L119.4,321.1 L119.9,320.9 L120.5,320.7
 L121.0,320.5 L121.6,320.4 L122.1,320.2 L122.7,320.0 L123.2,319.8 L123.8,319.6 L124.3,319.4 L124.9,319.2
 L125.4,319.0 L126.0,318.8 L126.5,318.6 L127.1,318.4 L127.6,318.2 L128.2,318.1 L128.7,317.9 L129.3,317.7
 L129.8,317.5 L130.4,317.3 L130.9,317.1 L131.5,316.9 L132.0,316.7 L132.6,316.6 L133.1,316.4 L133.7,316.2
 L134.2,316.0 L134.8,315.8 L135.3,315.6 L135.9,315.5 L136.4,315.3 L137.0,315.1 L137.5,314.9 L138.1,314.7
 L138.6,314.6 L139.2,314.4 L139.7,314.2 L140.3,314.0 L140.8,313.8 L141.4,313.7 L141.9,313.5 L142.5,313.3
 L143.0,313.1 L143.6,313.0 L144.1,312.8 L144.7,312.6 L145.2,312.4 L145.8,312.3 L146.3,312.1 L146.9,311.9
 L147.4,311.7 L148.0,311.6 L148.5,311.4 L149.1,311.2 L149.6,311.0 L150.2,310.9 L150.7,310.7 L151.3,310.5
 L151.8,310.3 L152.4,310.2 L152.9,310.0 L153.5,309.8 L154.0,309.7 L154.6,309.5 L155.1,309.3 L155.7,309.1
 L156.2,309.0 L156.8,308.8 L157.3,308.6 L157.9,308.5 L158.4,308.3 L159.0,308.1 L159.5,307.9 L160.1,307.8
 L160.6,307.6 L161.2,307.4 L161.7,307.3 L162.3,307.1 L162.8,306.9 L163.4,306.7 L163.9,306.6 L164.5,306.4
 L165.0,306.2 L165.6,306.1 L166.1,305.9 L166.7,305.7 L167.2,305.6 L167.8,305.4 L168.3,305.2 L168.9,305.0
 L169.4,304.9 L170.0,304.7 L170.5,304.5 L171.1,304.4 L171.6,304.2 L172.2,304.0 L172.7,303.9 L173.3,303.7
 L173.8,303.5 L174.4,303.3 L174.9,303.2 L175.5,303.0 L176.0,302.8 L176.6,302.7 L177.1,302.5 L177.7,302.3
 L178.2,302.2 L178.8,302.0 L179.3,301.8 L179.9,301.6 L180.4,301.5 L181.0,301.3 L181.5,301.1 L182.1,301.0
 L182.6,300.8 L183.2,300.6 L183.7,300.4 L184.3,300.3 L184.8,300.1 L185.4,299.9 L185.9,299.8 L186.5,299.6
 L187.0,299.4 L187.6,299.2 L188.1,299.1 L188.7,298.9 L189.2,298.7 L189.8,298.5 L190.3,298.4 L190.9,298.2
 L191.4,298.0 L192.0,297.8 L192.5,297.7 L193.1,297.5 L193.6,297.3 L194.2,297.1 L194.7,297.0 L195.3,296.8
 L195.8,296.6 L196.4,296.4 L196.9,296.2 L197.5,296.1 L198.0,295.9 L198.6,295.7 L199.1,295.5 L199.7,295.4
 L200.2,295.2 L200.8,295.0 L201.3,294.8 L201.9

Points of separation are obtained by finding the positions where the value of the gradient of velocityX normal to the wall changes its sign. 

In [ ]:
List<double> Rey = new List<double>();
List<double> Lenghths = new List<double>();

List<double> ReyTop = new List<double>();
List<double> Lenghths_left = new List<double>();
List<double> Lenghths_right = new List<double>();

foreach(var a in ReattachmentPositionsTOP){
    try{
        ReyTop.Add(a.Key.Item2);
        Lenghths_left.Add(a.Value[0]);
        Lenghths_right.Add(a.Value[1]);

        }catch(Exception e){
            Lenghths_left.Add(0);
            Lenghths_right.Add(0);
        continue;
        }
}
foreach(var a in ReattachmentPositions){
        double Reynolds = a.Key.Item2;
        double[] allLengths = a.Value;
        try{
            double primLength = allLengths[0];
            if(primLength < 0.3) //Try to filter values in order to obtain the length of reattachment after the aparition of the primary vortex  
                primLength = allLengths[1];
                
            Rey.Add(Reynolds);
            Lenghths.Add(primLength);
        }catch(Exception e){
        continue;
        }
}


Comparison of reattachment lengths for the primary vortex for ER =1.9423 

In [ ]:
var plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines;

fmt.LineColor = myCollors[0];
plot.AddDataGroup("BoSSS", Re_arr, LeArr, fmt);

// Reference data
MultidimensionalArray[] ReferenceDataNu = new MultidimensionalArray[1];

string path2 = Path.Combine(Directory.GetCurrentDirectory(), @"reatlengths_bot.txt"); // CF_RE400_ER1data for ER 1.94...
ReferenceDataNu[0] = IMatrixExtensions.LoadFromTextFile(path2);
var fmt4 = new PlotFormat();
fmt4.Style = Styles.Points;

fmt4.PointType = myPointTypes[0];// PointTypes.Diamond;
fmt4.PointSize = 0.5;
var xArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 0).To1DArray();
for(int i = 0; i < xArr_nu.Length; i++){
    xArr_nu[i]*=0.5; // REescale Reynolds for comparison with paper 
}

var yArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 1).To1DArray();
plot.AddDataGroup("reference",xArr_nu,yArr_nu,fmt4);
plot . PlotNow ()


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 -50 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 350 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 BoSSS 
 
 
 BoSSS 
 
 
 
 
 
 reference 
 
 
 reference

In [ ]:
var plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines;

fmt.LineColor = myCollors[0];
plot.AddDataGroup("BoSSS_Deatach",ReyTop.ToArray(),Lenghths_left.ToArray(), fmt);
plot.AddDataGroup("BoSSS_ReAtach",ReyTop.ToArray(),Lenghths_right.ToArray(), fmt);

// Reference data
MultidimensionalArray[] ReferenceDataLeft = new MultidimensionalArray[1];
string path2 = Path.Combine(Directory.GetCurrentDirectory(), @"reatlengths_top_left.txt"); // CF_RE400_ER1data for ER 1.94...
ReferenceDataLeft[0] = IMatrixExtensions.LoadFromTextFile(path2);

MultidimensionalArray[] ReferenceDataRight = new MultidimensionalArray[1];
string path = Path.Combine(Directory.GetCurrentDirectory(), @"reatlengths_top_right.txt"); // CF_RE400_ER1data for ER 1.94...
ReferenceDataRight[0] = IMatrixExtensions.LoadFromTextFile(path);


var fmt4 = new PlotFormat();
fmt4.Style = Styles.Points;
fmt4.PointType = myPointTypes[0];// PointTypes.Diamond;
fmt4.PointSize = 0.5;


var xArr_left = ReferenceDataLeft[0].ExtractSubArrayShallow(-1, 0).To1DArray();
var yArr_left = ReferenceDataLeft[0].ExtractSubArrayShallow(-1, 1).To1DArray();
for(int i = 0; i < xArr_left.Length; i++){
    xArr_left[i]*=0.5; // REescale Reynolds for comparison with paper 
}


var xArr_right = ReferenceDataRight[0].ExtractSubArrayShallow(-1, 0).To1DArray();
var yArr_right = ReferenceDataRight[0].ExtractSubArrayShallow(-1, 1).To1DArray();
for(int i = 0; i < xArr_right.Length; i++){
    xArr_right[i]*=0.5; // REescale Reynolds for comparison with paper 
}



plot.AddDataGroup("reference",xArr_left,yArr_left,fmt4);
plot.AddDataGroup("reference",xArr_right,yArr_right,fmt4);
plot.PlotNow ()


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 350 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 BoSSSDeatach 
 
 
 BoSSS D eatach 
 
 
 
 
 
 BoSSSReAtach 
 
 
 BoSSS R eAtach 
 
 
 
 
 
 reference 
 
 
 reference 
 
 
 
 
 
 
 
 
 
 
 
 
 reference 
 
 
 reference

In [ ]:
ReyTop

In [ ]:
Lenghths_left

In [ ]:

List<double> ReyTop = new List<double>();
List<double> Lenghths_left = new List<double>();
List<double> Lenghths_right = new List<double>();

In [ ]:
Lenghths_left_arr

In [ ]:
myDb.Sessions

#0: HBFS_ManualHomotopy_shorterDom	Heated_BackwardFacingStepdg2k10Re1000ER1.9423VPFalse*	03/04/2022 13:15:19	8b865428...
